In [5]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
%autoreload

import sys
import json
import glob
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

pd.set_option('max_colwidth', None)

sys.path.insert(0, '../../src-py/')

In [14]:
import moral_utils

In [15]:
data_path = '/home/sile2804/data-tmp/moral-debater-data'

In [16]:
def is_moral_concept(aspect, moral_concepts):
    aspect_words = aspect.split()
    for concept, moral in moral_concepts.items():
        concept_words = concept.split()
        overlap = len(set(aspect_words).intersection(concept_words))/len(concept_words)
        if overlap > 0.5:
            return moral
            
    return 'None'

In [53]:
data = []
for file in glob.glob(data_path + '/aspect-controlled-argument-generation/reddit_data/*/processed/merged.jsonl'):
    print(file)
    for line in open(file).readlines():
        json_obj = json.loads(line)
        json_obj['topic'] = file.split('/')[-3]
        data.append(json_obj)

/workspace/ceph_data/data/aspect-controlled-argument-generation/reddit_data/abortion/processed/merged.jsonl
/workspace/ceph_data/data/aspect-controlled-argument-generation/reddit_data/gun_control/processed/merged.jsonl
/workspace/ceph_data/data/aspect-controlled-argument-generation/reddit_data/marijuana_legalization/processed/merged.jsonl
/workspace/ceph_data/data/aspect-controlled-argument-generation/reddit_data/death_penalty/processed/merged.jsonl
/workspace/ceph_data/data/aspect-controlled-argument-generation/reddit_data/nuclear_energy/processed/merged.jsonl
/workspace/ceph_data/data/aspect-controlled-argument-generation/reddit_data/cloning/processed/merged.jsonl
/workspace/ceph_data/data/aspect-controlled-argument-generation/reddit_data/minimum_wage/processed/merged.jsonl
/workspace/ceph_data/data/aspect-controlled-argument-generation/reddit_data/school_uniforms/processed/merged.jsonl


In [59]:
df = pd.DataFrame(data)

In [60]:
df.head()

,id,doc_sent_id,stance,sent,doc_id,doc_metadata_id,doc_url,doc_score,index,aspect_string,aspect_pos
0,0,1,Argument_against,"However , I believe abortion is a health issue and abortion is part of the healthcare industry .",--1Tc2sBJVi6mb4YT3ns,c1p39fy,https://www.reddit.com//comments/go7wt//c1p39fy,11.047030,redditcomments-en,"[health issue, healthcare industry]","[[7, 8], [15, 16]]"
1,1,0,Argument_against,Because abortion forces everyone to have an abortion ! !,--6EhGsBJVi6mb4YCsro,c4hefg3,https://www.reddit.com//comments/suc58//c4hefg3,12.300539,redditcomments-en,[],[]
2,2,1,Argument_against,"Adoption is an *alternative* to abortion , not a solution .",--EB_GsBJVi6mb4Y_22f,czwafag,https://www.reddit.com//comments/459xmn//czwafag,10.695252,redditcomments-en,"[Adoption, solution]","[[0, 0], [9, 9]]"
3,3,0,Argument_for,And yet abortion is legal in this country .,--EXzWsBJVi6mb4Yx55U,d5utytm,https://www.reddit.com//comments/4v1he8//d5utytm,10.691054,redditcomments-en,"[abortion, legal]","[[2, 2], [4, 4]]"
4,4,2,Argument_against,"Why ca n't she see the obvious , that abortion is a very private decision for the woman and the provider , no one else .",--HamGwBJVi6mb4Y4B64,e8r7j8p,https://www.reddit.com/r/politics/comments/9ssif9/antiabortion_conservative_running_for_texas/e8r7j8p/,10.326312,redditcomments-en,[private decision],"[[13, 14]]"


In [61]:
moral_concepts = moral_utils.get_moral_concepts(preprocess=True)
concept_to_morals = {c:moral for moral in moral_concepts for c in moral_concepts[moral]}

In [62]:
df['moral_frame'] = df['aspect_string'].apply(lambda aspects: [is_moral_concept(aspect, concept_to_morals) for aspect in aspects])

In [65]:
df['moral_frame'] = df['moral_frame'].apply(lambda aspects: set([a for a in aspects if a != 'None']))
#adjust
#df['moral_frame'] = df['moral_frame'].apply(lambda aspects: aspects.remove('care') if len(aspects) > 2 else aspects)

In [66]:
df.head()

,id,doc_sent_id,stance,sent,doc_id,doc_metadata_id,doc_url,doc_score,index,aspect_string,aspect_pos,moral_frame
0,0,1,Argument_against,"However , I believe abortion is a health issue and abortion is part of the healthcare industry .",--1Tc2sBJVi6mb4YT3ns,c1p39fy,https://www.reddit.com//comments/go7wt//c1p39fy,11.047030,redditcomments-en,"[health issue, healthcare industry]","[[7, 8], [15, 16]]",{}
1,1,0,Argument_against,Because abortion forces everyone to have an abortion ! !,--6EhGsBJVi6mb4YCsro,c4hefg3,https://www.reddit.com//comments/suc58//c4hefg3,12.300539,redditcomments-en,[],[],{}
2,2,1,Argument_against,"Adoption is an *alternative* to abortion , not a solution .",--EB_GsBJVi6mb4Y_22f,czwafag,https://www.reddit.com//comments/459xmn//czwafag,10.695252,redditcomments-en,"[Adoption, solution]","[[0, 0], [9, 9]]",{}
3,3,0,Argument_for,And yet abortion is legal in this country .,--EXzWsBJVi6mb4Yx55U,d5utytm,https://www.reddit.com//comments/4v1he8//d5utytm,10.691054,redditcomments-en,"[abortion, legal]","[[2, 2], [4, 4]]",{authority}
4,4,2,Argument_against,"Why ca n't she see the obvious , that abortion is a very private decision for the woman and the provider , no one else .",--HamGwBJVi6mb4Y4B64,e8r7j8p,https://www.reddit.com/r/politics/comments/9ssif9/antiabortion_conservative_running_for_texas/e8r7j8p/,10.326312,redditcomments-en,[private decision],"[[13, 14]]",{}


In [56]:
df.to_pickle(data_path + '/aspect-controlled-argument-generation/reddit_data_with_morals.pkl')

In [13]:
df = pd.read_pickle(data_path + '/aspect-controlled-argument-generation/reddit_data_with_morals.pkl')

In [14]:
df.head()

,id,doc_sent_id,stance,sent,doc_id,doc_metadata_id,doc_url,doc_score,index,aspect_string,aspect_pos,moral_frame,topic
3,3,0,Argument_for,And yet abortion is legal in this country .,--EXzWsBJVi6mb4Yx55U,d5utytm,https://www.reddit.com//comments/4v1he8//d5utytm,10.691054,redditcomments-en,"[abortion, legal]","[[2, 2], [4, 4]]",authority,abortion
5,5,0,Argument_against,My favorite is calling the restriction on abortions a human rights crisis .,--N-1GsBJVi6mb4YFmkU,c5yo16l,https://www.reddit.com//comments/ytay3//c5yo16l,9.626792,redditcomments-en,[human rights crisis],"[[9, 11]]",fairness,abortion
6,6,1,Argument_for,"Also , I believe in England and Wales they have about 79 abortions performed per year to save the life of the mother .",--N-1GsBJVi6mb4YFmkU,c5yo16l,https://www.reddit.com//comments/ytay3//c5yo16l,9.626792,redditcomments-en,[save the life of the mother],"[[17, 22]]",authority,abortion
7,7,3,Argument_for,"( Besides , life of the mother is almost always an acceptable reason for abortion , as long as aborting the baby is a secondary effect . )",--N-1GsBJVi6mb4YFmkU,c5yo16l,https://www.reddit.com//comments/ytay3//c5yo16l,9.626792,redditcomments-en,"[life of the mother, acceptable reason, secondary]","[[3, 6], [11, 12], [24, 24]]",authority,abortion
14,14,0,Argument_against,& gt ; But abortion is a sin ... but this story is made up ... .,--VSEGwBJVi6mb4YmjP2,cy9cylr,https://www.reddit.com//comments/3xz757//cy9cylr,10.120193,redditcomments-en,"[abortion, sin]","[[4, 4], [7, 7]]",purity,abortion


In [15]:
df[df.moral_frame.str.len() > 0].count()

id                 910279
doc_sent_id        910279
stance             910279
sent               910279
doc_id             910279
doc_metadata_id    910279
doc_url            910279
doc_score          910279
index              910279
aspect_string      910279
aspect_pos         910279
moral_frame        910279
topic              910279
dtype: int64

In [19]:
df[df.moral_frame.str.len() == 1].moral_frame.value_counts()

TypeError: unhashable type: 'set'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'set'


{care}         368604
{authority}    350808
{purity}        78311
{fairness}      66161
{loyalty}       46395
Name: moral_frame, dtype: int64

In [10]:
df = df[df.moral_frame.str.len() == 1]
df.moral_frame = df.moral_frame.apply(lambda x: x.pop())

In [16]:
df.moral_frame.value_counts()

care         368604
authority    350808
purity        78311
fairness      66161
loyalty       46395
Name: moral_frame, dtype: int64

In [63]:
def sampling_k_elements(group, k=46395):
    if len(group) < k:
        return group
    return group.sample(k)

balance_df = df.groupby('moral_frame').apply(sampling_k_elements).reset_index(drop=True)

In [64]:
balance_df.head()

,id,doc_sent_id,stance,sent,doc_id,doc_metadata_id,doc_url,doc_score,index,aspect_string,aspect_pos,moral_frame,topic
0,65266,2,Argument_against,"It does n't help that you ca n't advocate legalization to the people who need to understand about it without sounding like a huge stoner , either .",8PjpJGwBJVi6mb4Y-VX7,c06v3xf,https://www.reddit.com//comments/7jqbd//c06v3xf,34.614014,redditcomments-en,[advocate legalization],"[[8, 9]]",authority,marijuana_legalization
1,145785,1,Argument_against,I 'm mocking the American gun nuts who point to the UK and say that gun control laws have no effect .,EsY3EmwBJVi6mb4YMyiK,ch0y705,https://www.reddit.com//comments/23vcf7//ch0y705,28.958755,redditcomments-en,[gun control laws],"[[15, 17]]",authority,gun_control
2,122467,0,Argument_against,Seems a little silly when you could probably just order hundreds of them or clones thereof from China .,soePDWwBJVi6mb4Y9_TP,d9ttl52,https://www.reddit.com//comments/5c2f6d//d9ttl52,8.481560,redditcomments-en,"[order, clones]","[[9, 9], [14, 14]]",authority,cloning
3,165829,0,Argument_against,"I think it 's insane that religion still has so much control in the way we view things , especially when it comes to abortion and marriage ( the gay marriage debate for example ) I 've never read the Bible but I 'm sure it does n't say anywhere that you ca n't abort a child ( correct me if I 'm wrong ) and if it did , in the 21st century we should n't be letting that book dictate personal choice ...",BSsy8GsBJVi6mb4YYSI_,dr17riu,https://www.reddit.com/r/worldnews/comments/7iqscd/salvadoran_woman_has_been_jailed_for_decade_since/dr17riu/,7.323644,redditcomments-en,"[religion, control, dictate, personal choice]","[[6, 6], [11, 11], [81, 81], [82, 83]]",authority,abortion
4,261204,1,Argument_against,Not only are they making it a crime to cross state lines for an abortion - which is a fundamental infringement on a person 's right to travel - it 's also a crime to help a person cross state lines .,Ud3WUG0BJVi6mb4YDjaF,eonaxsd,https://www.reddit.com/r/Feminism/comments/bshhyb/interesting_how_some_constitutional_rights_are/eonaxsd/,7.329508,redditcomments-en,"[crime, cross state lines, fundamental infringement, right to travel, crime, state lines]","[[7, 7], [9, 11], [19, 20], [25, 27], [33, 33], [39, 40]]",authority,abortion


In [67]:
balance_df.moral_frame.value_counts()

fairness     46395
purity       46395
care         46395
loyalty      46395
authority    46395
Name: moral_frame, dtype: int64

In [2]:
len(balance_df)

NameError: name 'balance_df' is not defined

In [68]:
balance_df.topic.value_counts()

gun_control               44953
death_penalty             35502
marijuana_legalization    33254
minimum_wage              32238
abortion                  27411
nuclear_energy            26274
cloning                   16520
school_uniforms           15823
Name: topic, dtype: int64

In [69]:
train_df = balance_df[balance_df.topic.isin(['gun_control', 'marijuana_legalization', 'abortion', 'nuclear_energy', 'death_penalty', 'minimum_wage'])]
test_df  = balance_df[balance_df.topic.isin(['cloning', 'school_uniforms'])]

In [70]:
train_df.to_pickle(data_path + '/aspect-controlled-argument-generation/reddit_data_with_morals_train.pkl')
test_df.to_pickle(data_path + '/aspect-controlled-argument-generation/reddit_data_with_morals_test.pkl')